### bq 데이터(pandas)를 redis 서버에 저장

In [9]:
from pydatafabric.vault_utils import get_secrets

PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_data"
DT = "2022-08-30"
LIMIT = 30

REDIS_INFO = get_secrets(mount_point="datafabric", path="redis/datafabric/dev")

In [10]:
from pydatafabric.gcp import bq_to_pandas


df = bq_to_pandas(f"""
    select  *
    from  `{PROJECT}.{DATASET}.{TABLE}`
    WHERE dt = '{DT}'
    limit {LIMIT}
""")

destination: emart-datafabric._2dd36219768c7c869a5680edf9fd6e104ea57800.anon6184b5484ab1de1a15dc07fe884c8155a72509f19c05725907277d9b4861d4d7
total_rows: 30
slot_secs: 0.084



Downloading: 100%|██████████| 30/30 [00:00<00:00, 34.29rows/s]


In [12]:
df.head()

,review_id,cust_id,gender_cd,gender_nm,age,agrde_cd_10_unit,store_cd,store_nm,score,comments,...,longitude,latitude,comments_point,image_point,thumb_point,action_cd,blind_flag,active_flag,tag_list,avg_tag_score
0,2208301432069315,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,치즈 좋아하는데 쟁여 놔야 맘이 편안 그래서 또구입,...,127.046690683,37.651608368,10,10,0,003,N,Y,"332::1::8,332::2::6,332::3::6,332::4::6,332::5::6",6.000000000
1,2208301435131475,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,모짜렐라 쟁여 놓은게 마침 떨어져서 구입 든든,...,127.046690683,37.651608368,10,10,0,003,N,Y,"332::1::6,332::2::6,332::3::6,332::4::6,332::5::4",6.000000000
2,2208301430288840,02b7d95c81e0a0b16905e77547ac0b54881a0728438961...,2,여,62,60,1000,이마트 창동점,10,담백하고 비리지 않은 가자미 국산이라네요 맛있게 먹을께용,...,127.046690683,37.651608368,10,10,0,003,N,Y,"153::1::6,153::2::6,153::3::6,153::4::6,153::5::6",6.000000000
3,2208300923271031,03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4...,1,남,36,30,1000,이마트 창동점,10,저렴하고 맛있어요!!!!!!!!!,...,127.046690683,37.651608368,10,10,0,003,N,Y,"130::1::10,130::2::10,130::3::10,130::4::10,13...",10.000000000
4,2208300925053511,03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4...,1,남,36,30,1000,이마트 창동점,10,맛있어요!!!!!!!!!,...,127.046690683,37.651608368,10,10,0,003,N,Y,"340::1::10,340::2::10,340::3::10,340::4::10,34...",10.000000000


In [13]:
print(df.dtypes)

review_id           object
cust_id             object
gender_cd           object
gender_nm           object
age                 object
agrde_cd_10_unit    object
store_cd            object
store_nm            object
score                Int64
comments            object
comments_length      Int64
sku_cd              object
sku_nm              object
order_dt            dbdate
dt                  dbdate
prdt_cat_cd         object
prdt_cat_nm         object
prdt_di_cd          object
prdt_di_nm          object
prdt_gcode_cd       object
prdt_gcode_nm       object
prdt_mcode_cd       object
prdt_mcode_nm       object
prdt_dcode_cd       object
prdt_dcode_nm       object
area_cd             object
area_nm             object
longitude           object
latitude            object
comments_point       Int64
image_point          Int64
thumb_point          Int64
action_cd           object
blind_flag          object
active_flag         object
tag_list            object
avg_tag_score       object
d

In [17]:
import redis
import json

# 레디스 연결
rd = redis.StrictRedis(host=REDIS_INFO["host"], port=REDIS_INFO["port"], db=0)


if df["review_id"].count() > 0:
    json_df = df.to_json()
    rd.set("test_dfm_sample_eapp_data", json_df)

In [18]:
result = rd.get("test_dfm_sample_eapp_data")
result = result.decode('utf-8')
result = dict(json.loads(result))

print(result)

{'review_id': {'0': '2208301432069315', '1': '2208301435131475', '2': '2208301430288840', '3': '2208300923271031', '4': '2208300925053511', '5': '2208300924173039', '6': '2208300929267098', '7': '2208300930213869', '8': '2208300928379733', '9': '2208301348410479', '10': '2208301346346704', '11': '2208300828497973', '12': '2208300006452516', '13': '2208300005065291', '14': '2208300002085208', '15': '2208301130070273', '16': '2208301127350498', '17': '2208301126087760', '18': '2208301124312080', '19': '2208301131198831', '20': '2208301132341431', '21': '2208301135170682', '22': '2208301122560242', '23': '2208301122086191', '24': '2208301136446921', '25': '2208301532413695', '26': '2208301525149903', '27': '2208301530516339', '28': '2208301522476839', '29': '2208301523250645'}, 'cust_id': {'0': '02b7d95c81e0a0b16905e77547ac0b54881a0728438961c0524fc6666c200663', '1': '02b7d95c81e0a0b16905e77547ac0b54881a0728438961c0524fc6666c200663', '2': '02b7d95c81e0a0b16905e77547ac0b54881a0728438961c052